In [7]:
import re
f = open('dataset/bing_liu/Nikon coolpix 4300.txt', 'r')

pattern_title = '\[t\]'
pattern_sentence = '(?<=##).+'
pattern_aspect = '.+(?=##)'
pattern_aspect_anomali = '\[(u|p|s|cc|cs)\]'

sentences = []
review = []
count = 0
for a in f:
    if re.search('##', a):
        sentence = re.findall(pattern_sentence, a)[0]
        aspect = re.findall(pattern_aspect, a)
#         anomali = re.findall(pattern_aspect_anomali, a)
#         if anomali:
#             print(sentence, aspect)
#         else:
        if len(aspect) > 0:
            aspect = aspect[0]
        else:
            aspect = ''
        sentences.append((sentence, aspect))

In [8]:
import pandas as pd

df = pd.DataFrame(columns=['review','aspect'])
for r in sentences:
    df = df.append({'review': r[0], 'aspect': r[1]}, ignore_index=True)
df.to_csv("nikon.csv")

In [2]:
import pandas as pd
df = pd.read_csv('res_mul_all.csv')

f = open("data/restaurant.txt","w+")
for i in df['review']:
     f.write(i + "\n")
f.close()

In [38]:
import pandas as pd
import xml.etree.ElementTree as et

def parse_restaurant(dataset):
    df_cols = ["reviewID", "sentenceID", "review", "target", "category", "polarity"]
    out_df = pd.DataFrame(columns = df_cols)

    isMultipleCategories = True
    xtree_train = et.parse(dataset)
    xroot_train = xtree_train.getroot()

    for node in xroot_train: 
        rid = node.attrib.get('rid')
        sentences = node.find('sentences') if node is not None else None
        for sentence in sentences:
            target = []
            categories = []
            polarities = []
            sid = sentence.attrib.get('id')
            oos = sentence.attrib.get('OutOfScope')
            if oos == "TRUE":
                continue
            text = sentence.find('text').text if sentence is not None else None
            opinions = sentence.find('Opinions') if sentence is not None else []
            if opinions is not None:
                for opinion in opinions:
                    target.append(opinion.attrib.get('target'))
                    categories.append(opinion.attrib.get('category'))
                    polarities.append(opinion.attrib.get('polarity'))
            else:
                continue
            
            out_df = out_df.append(pd.Series([rid,
                                              sid,
                                              text,
                                              ','.join(target),
                                              ','.join(categories),
                                              ','.join(polarities)],
                                             index = df_cols), ignore_index = True)

    return out_df

In [39]:
sf = parse_restaurant("dataset/ABSA16/ABSA16_Restaurants_Train_SB1_v2.xml")

NULL
SERVICE#GENERAL

NULL
SERVICE#GENERAL

NULL,NULL
RESTAURANT#PRICES,FOOD#QUALITY

NULL
SERVICE#GENERAL

NULL
RESTAURANT#GENERAL

NULL,service,decor
FOOD#QUALITY,SERVICE#GENERAL,AMBIENCE#GENERAL

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
SERVICE#GENERAL

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#MISCELLANEOUS

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
FOOD#QUALITY

NULL
RESTAURANT#MISCELLANEOUS

NULL
RESTAURANT#GENERAL

NULL
AMBIENCE#GENERAL

NULL
RESTAURANT#GENERAL

NULL,NULL
FOOD#QUALITY,FOOD#PRICES

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
FOOD#STYLE_OPTIONS

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
SERVICE#GENERAL

NULL
RESTAURANT#GENERAL

NULL,toppings
FOOD#PRICES,FOOD#PRICES

NULL
RESTAURANT#GENERAL

NULL
SERVICE#GENERAL

NULL,NULL
RESTAURANT#PRICES,RESTAURANT#GENERAL

NULL
AMBIENCE#GENERAL

NULL
RESTAURANT#GENERAL

NULL
FOOD#

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL,NULL
FOOD#PRICES,FOOD#QUALITY

NULL,fish and chips
FOOD#PRICES,FOOD#PRICES

NULL
FOOD#QUALITY

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL,NULL
FOOD#QUALITY,FOOD#STYLE_OPTIONS

NULL,NULL
FOOD#PRICES,FOOD#QUALITY

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
FOOD#QUALITY

NULL
SERVICE#GENERAL

NULL
RESTAURANT#MISCELLANEOUS

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
RESTAURANT#GENERAL

NULL
FOOD#QUALITY

NULL
FOOD#QUALITY

NULL
RESTAURANT#GENERAL

NULL
DRINKS#QUALITY

NULL
RESTAURANT#GENERAL

NULL
SERVICE#GENERAL

NULL,NULL,food,NULL
RESTAURANT#GENERAL,AMBIENCE#GENERAL,FOOD#QUALITY,RESTAURANT#MISCELLANEOUS

In [40]:
sf.head()

,reviewID,sentenceID,review,target,category,polarity
0,1004293,1004293:0,Judging from previous posts this used to be a ...,place,RESTAURANT#GENERAL,negative
1,1004293,1004293:1,"We, there were four of us, arrived at noon - t...",staff,SERVICE#GENERAL,negative
2,1004293,1004293:2,"They never brought us complimentary noodles, i...",NULL,SERVICE#GENERAL,negative
3,1004293,1004293:3,The food was lousy - too sweet or too salty an...,"food,portions","FOOD#QUALITY,FOOD#STYLE_OPTIONS","negative,negative"
4,1004293,1004293:4,"After all that, they complained to me about th...",NULL,SERVICE#GENERAL,negative


In [19]:
sf.to_csv("dataset/res16.csv")